# Fine-tuning a pre-trained model

The ability to adapt pre-trained models to specific tasks has revolutionized how we approach complex language problems. BERT (Bidirectional Encoder Representations from Transformers) has emerged as a particularly powerful model for a wide range of NLP tasks. The process of adapting a pre-trained model to a specific task is known as fine-tuning. Fine-tuning allows us to leverage the model's learned representations, adjusting them slightly with additional training on a smaller, task-specific dataset. 

### Objective
In this notebook, we illustrate the process of fine-tuning a pre-trained BERT model for a sentiment analysis task. We aim to demonstrate the effectiveness of fine-tuning by comparing the performance of the pre-trained model before and after fine-tuning on a small custom dataset. The task involves classifying text into two categories: positive and negative sentiment. Through this exercise, we seek to achieve two main goals:

1. Showcase the Process of Fine-Tuning: We will walk through the steps required to fine-tune a pre-trained BERT model on a custom sentiment analysis dataset. This includes data preparation, model configuration, training, and evaluation.
2. Demonstrate the Impact of Fine-Tuning: By comparing the model's performance on our task before and after fine-tuning, we can observe the improvements that can be achieved through this process.

#### Import Libraries

In [18]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd

### Creating a Custom Dataset for Sentiment Analysis
First, we construct a small dataset for demonstration. This dataset consists of texts labeled for sentiment: 1 for positive and 0 for negative. Normally, you'd use a more extensive dataset for robust model training.


In [19]:
data = {
    'text': [
        'I love this product!',
        'Absolutely wonderful service.',
        'Not what I expected, sadly.',
        'The experience was bad, very bad.',
        'Fantastic! Will come again.',
        'Do not recommend.',
        'Great value for the money.',
        'Worst purchase I ever made.',
        'Happy with my purchase!',
        'Terrible, I hated it.'
    ],
    'label': [1, 1, 0, 0, 1, 0, 1, 0, 1, 0]
}

df = pd.DataFrame(data)
train_df = df.sample(frac=0.8, random_state=200) # 80% for training
val_df = df.drop(train_df.index) # 20% for validation

The above creates a pandas DataFrame from a dictionary. We then split this DataFrame into training and validation sets, ensuring that the model sees only a portion of the data during training, which helps evaluate its performance on unseen data.



### Preprocessing the Data
Preprocessing involves tokenizing the text data, converting it into a format BERT understands. This includes adding special tokens, padding, and creating attention masks. This code snippet prepares our dataset for training by tokenizing the text, padding sequences to a uniform length, and creating attention masks to help the model distinguish between content and padding.

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def convert_examples_to_features(text, label):
    # Tokenize and encode sentences in the BERT's way
    input_ids, attention_masks = [], []
    for t in text:
        encoded = tokenizer.encode_plus(t, add_special_tokens=True, max_length=128, padding='max_length', truncation=True, return_attention_mask=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant(input_ids), "attention_mask": tf.constant(attention_masks)}, tf.constant(label))).batch(2)

# Preparing dataset for training and validation
train_data = convert_examples_to_features(train_df['text'].values, train_df['label'].values)
val_data = convert_examples_to_features(val_df['text'].values, val_df['label'].values)

### Evaluate Pre-trained Model Performance
Before fine-tuning, we evaluate the pre-trained BERT model on our validation set to establish a performance baseline.

In [23]:
# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

# Evaluate the pre-trained model
baseline_result = model.evaluate(val_data)
print(f"Baseline accuracy (pre-trained): {baseline_result[1]*100:.2f}%")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1/1 [==============================] - 3s 3s/step - loss: 0.7474 - accuracy: 0.5000
Baseline accuracy (pre-trained): 50.00%


### Fine-Tune BERT on the Custom Dataset
Now, we'll fine-tune the model on our training data and evaluate it again.

In [24]:
# Fine-tune the model
model.fit(train_data, epochs=2, validation_data=val_data)

# Evaluate the fine-tuned model
finetuned_result = model.evaluate(val_data)
print(f"Accuracy after fine-tuning: {finetuned_result[1]*100:.2f}%")

Epoch 1/2
4/4 [==============================] - 18s 1s/step - loss: 0.7076 - accuracy: 0.6250 - val_loss: 0.5528 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 189ms/step - loss: 0.4668 - accuracy: 1.0000
Accuracy after fine-tuning: 100.00%


### Compare Results
We compare the baseline accuracy of the pre-trained model with the accuracy after fine-tuning.

In [25]:
improvement = finetuned_result[1] - baseline_result[1]
print(f"Improvement in accuracy: {improvement*100:.2f}%")

Improvement in accuracy: 50.00%


The fine-tuning process adapts the pre-trained BERT model to our specific sentiment analysis task. By comparing the model's accuracy before and after fine-tuning, we can observe the effect of this adaptation. The fine-tuned model showed an improved accuracy on the validation set, demonstrating the value of fine-tuning for custom NLP tasks.